In [86]:
import utils
from nuesslein1 import nuesslein1
from nuesslein2 import nuesslein2
from chancellor import chancellor
from choi import choi
import re
import os
import numpy as np
from dimod.utilities import qubo_to_ising

In [85]:
N = 40
M = 170
formula = utils.create_formula(N, M, 3)
formula

[array([28, -8, -3]),
 array([-33,  13, -40]),
 array([ 20,  -9, -27]),
 array([ 12, -33,  37]),
 array([-14, -28,  16]),
 array([-39,   4, -23]),
 array([ 16, -34,  20]),
 array([ 39, -29,   4]),
 array([ 36, -19, -20]),
 array([ 24,  14, -17]),
 array([ -2, -40,  24]),
 array([-14,  -5,  33]),
 array([40, 30, 17]),
 array([  6,   4, -28]),
 array([-23,  20, -38]),
 array([36, 24,  1]),
 array([ 17, -14,  -8]),
 array([-23,  -3,  27]),
 array([-2, 13, 14]),
 array([ 36, -16, -38]),
 array([-24, -30,  38]),
 array([-14, -26,   1]),
 array([ -3, -23, -29]),
 array([ 23, -34,  -1]),
 array([  5,  31, -34]),
 array([ 17, -29,  10]),
 array([-20,  -5, -33]),
 array([-16,  39,  30]),
 array([ 5, 37, -3]),
 array([-33,  18,  -7]),
 array([  8, -25,  21]),
 array([ 25,  22, -19]),
 array([-37,  -6,  -9]),
 array([ 30,   8, -40]),
 array([25, 39, 13]),
 array([-31, -38,  22]),
 array([ -7,  14, -19]),
 array([ 40, -12, -37]),
 array([  7, -21, -10]),
 array([-25,  13,  -7]),
 array([-33,  -5, 

In [72]:
chanc = chancellor(formula=formula, V=N)
chanc.fillQ()
qubo_to_ising(chanc.Q)[0]


{0: 8.0,
 15: 2.0,
 29: -2.0,
 40: 2.0,
 16: -12.0,
 14: -2.0,
 28: 4.0,
 41: -2.0,
 4: -10.0,
 42: -2.0,
 18: 4.0,
 2: 0.0,
 31: 4.0,
 43: 2.0,
 32: -2.0,
 12: -4.0,
 11: 2.0,
 44: -2.0,
 20: 0.0,
 38: -4.0,
 45: -2.0,
 5: 6.0,
 33: -6.0,
 46: -2.0,
 23: 6.0,
 47: 2.0,
 37: 6.0,
 19: -2.0,
 3: 6.0,
 48: 2.0,
 17: 0.0,
 9: -6.0,
 49: -2.0,
 21: 8.0,
 50: 2.0,
 51: -2.0,
 52: 2.0,
 6: -2.0,
 53: -2.0,
 13: -12.0,
 1: 0.0,
 54: 2.0,
 36: -6.0,
 55: -2.0,
 24: 2.0,
 56: -2.0,
 57: -2.0,
 27: -8.0,
 58: -2.0,
 10: -2.0,
 7: -2.0,
 59: -2.0,
 60: 2.0,
 35: 0.0,
 61: 2.0,
 62: 2.0,
 63: -2.0,
 64: 2.0,
 30: -8.0,
 65: -2.0,
 22: 4.0,
 66: -2.0,
 67: 2.0,
 68: 2.0,
 39: 2.0,
 69: -2.0,
 70: 2.0,
 8: 0.0,
 71: -2.0,
 25: 4.0,
 72: -2.0,
 73: 2.0,
 74: 2.0,
 75: 2.0,
 76: -2.0,
 77: -2.0,
 78: 2.0,
 79: 2.0,
 80: -2.0,
 81: 2.0,
 82: 2.0,
 83: 2.0,
 84: 2.0,
 26: -2.0,
 85: -2.0,
 86: -2.0,
 87: 2.0,
 34: 4.0,
 88: -2.0,
 89: -2.0,
 90: 2.0,
 91: 2.0,
 92: 2.0,
 93: 2.0,
 94: 2.0,
 95: -2.0,
 9

In [91]:
name_map = {
    'nuesslein_2n': (2, nuesslein1),
    'nuesslein_n': (1, nuesslein2),
    'chancellor_n': (1, chancellor)
}
outpath = '/home/matt/Documents/benchmarks/CUSTOM_SAT/SAT_TO_ISING/cust-u40-170/'
for cnf in os.listdir(
    os.environ['BENCH']+'/CUSTOM_SAT/cust-u40-170'):
    with open( os.environ['BENCH']+'/CUSTOM_SAT/cust-u40-170/' + cnf) as cnffile:
        formula = []
        N, M = re.findall(r'p cnf (\d+) (\d+)', cnffile.readline())[0]
        N = int(N)
        M = int(M)
        for line in cnffile.readlines():
            args = line.strip().split();
            if (len(args) < 4):
                continue
            l1, l2, l3, _ = args
            formula.append(np.array([int(l1), int(l2), int(l3)]))
    cnf_base = cnf.replace('.cnf', '{}.gset')
    for name, (factor, func) in name_map.items():
        with open(cnf_base.format(name), mode='w') as testfile:
            obj = func(formula=formula, V=N)
            obj.fillQ()
            linear, quad, c = qubo_to_ising(chanc.Q)
            print(name, c)
            e = len(linear) + len(quad) 
            nodes = factor * N + M + 1
            testfile.write(f'{nodes} {e}\n')
            for (u,v), w in quad.items():
                testfile.write(f'{u} {v} {w}\n')
            
            for u, w in linear.items():
                testfile.write(f'{nodes-1} {u} {w}\n')


nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslein_2n -7664.0
nuesslein_n -7664.0
chancellor_n -7664.0
nuesslei

In [82]:
spinstate = "0 0 1 1 1 0 0 1 1 1 1 1 1 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 0 0 1 0 1 0 1 0 1 0 0 0 0 0 1 1 0 1 0 0 1 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 0 1 1 0 0 1 1 0 1 0 0 1 1 0 0 0 1 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1"
spins = [int(i) for i in spinstate.strip().split()]

In [83]:
utils.check_solution(formula, spins)

170